<a href="https://colab.research.google.com/github/kutluhanNG/MachineLearning/blob/main/Variational_Autoencoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. VA's are *probabilistic autoencoders* meaaning that their outputs are partly determined by chance, even after training.
2. Most importantly, they are *generative autoencoders* meaning they can generate new instances that look like they were sampled from the training set.



In [2]:
import tensorflow as tf
class Sampling(tf.keras.layers.Layer):
  def call(self, inputs):
    mean, log_var = inputs
    return tf.random.normal(tf.shape(log_var)) * tf.exp(log_var / 2) + mean

In [6]:
from tensorflow.keras.datasets import fashion_mnist
import numpy as np

# 1. Load the Fashion MNIST dataset.
# The dataset returns training and test sets.
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# 2. Preprocess the images by converting them to float32 and normalizing to [0, 1].
X_train = X_train.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

# 3. Create a validation set from the training data.
# For example, we can take the first 5000 samples as the validation set.
X_valid = X_train[:5000]
y_valid = y_train[:5000]
X_train = X_train[5000:]
y_train = y_train[5000:]

print("Training data shape:", X_train.shape)
print("Validation data shape:", X_valid.shape)
print("Test data shape:", X_test.shape)

Training data shape: (55000, 28, 28)
Validation data shape: (5000, 28, 28)
Test data shape: (10000, 28, 28)


In [9]:
codings_size = 10

inputs = tf.keras.layers.Input(shape=[28, 28])
Z = tf.keras.layers.Flatten()(inputs)
Z = tf.keras.layers.Dense(150, activation="relu")(Z)
Z = tf.keras.layers.Dense(100, activation="relu")(Z)
codings_mean = tf.keras.layers.Dense(codings_size)(Z)
codings_log_var = tf.keras.layers.Dense(codings_size)(Z)
codings = Sampling()([codings_mean, codings_log_var])
variational_encoder = tf.keras.Model(inputs=[inputs], outputs=[codings_mean, codings_log_var, codings])

decoder_inputs = tf.keras.layers.Input(shape=[codings_size])
x = tf.keras.layers.Dense(100, activation="relu")(decoder_inputs)
x = tf.keras.layers.Dense(150, activation="relu")(x)
x = tf.keras.layers.Dense(28 * 28)(x)
outputs = tf.keras.layers.Reshape([28, 28])(x)
variational_decoder = tf.keras.Model(inputs=[decoder_inputs], outputs=[outputs])

_, _, codings = variational_encoder(inputs)
reconstructions = variational_decoder(codings)
variational_ae = tf.keras.Model(inputs=[inputs], outputs=[reconstructions])

latent_loss = -0.5 * tf.reduce_sum(1 + codings_log_var - tf.exp(codings_log_var) - tf.square(codings_mean), axis=-1)
variational_ae.add_loss(tf.reduce_mean(latent_loss) / 784.)

variational_ae.compile(loss="mse", optimizer="nadam")
history = variational_ae.fit(X_train, X_train, epochs=50, batch_size=128, validation_data=(X_valid, X_valid))


ValueError: A KerasTensor cannot be used as input to a TensorFlow function. A KerasTensor is a symbolic placeholder for a shape and dtype, used when constructing Keras Functional models or Keras Functions. You can only use it as input to a Keras layer or a Keras operation (from the namespaces `keras.layers` and `keras.operations`). You are likely doing something like:

```
x = Input(...)
...
tf_fn(x)  # Invalid.
```

What you should do instead is wrap `tf_fn` in a layer:

```
class MyLayer(Layer):
    def call(self, x):
        return tf_fn(x)

x = MyLayer()(x)
```


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Custom Sampling layer.
class Sampling(layers.Layer):
    def call(self, inputs):
        mean, log_var = inputs
        epsilon = tf.random.normal(tf.shape(mean))
        return mean + tf.exp(0.5 * log_var) * epsilon

# Encoder definition.
codings_size = 10
encoder_inputs = layers.Input(shape=(28, 28))
x = layers.Flatten()(encoder_inputs)
x = layers.Dense(150, activation="relu")(x)
x = layers.Dense(100, activation="relu")(x)
codings_mean = layers.Dense(codings_size)(x)
codings_log_var = layers.Dense(codings_size)(x)
codings = Sampling()([codings_mean, codings_log_var])
encoder = Model(encoder_inputs, [codings_mean, codings_log_var, codings], name="encoder")

# Decoder definition.
decoder_inputs = layers.Input(shape=(codings_size,))
x = layers.Dense(100, activation="relu")(decoder_inputs)
x = layers.Dense(150, activation="relu")(x)
x = layers.Dense(28 * 28, activation="sigmoid")(x)
decoder_outputs = layers.Reshape((28, 28))(x)
decoder = Model(decoder_inputs, decoder_outputs, name="decoder")

# Define the VAE as a subclass of Model.
class VAE(Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def call(self, inputs):
        codings_mean, codings_log_var, codings = self.encoder(inputs)
        reconstruction = self.decoder(codings)
        # Compute the KL divergence loss.
        kl_loss = -0.5 * tf.reduce_sum(
            1 + codings_log_var - tf.square(codings_mean) - tf.exp(codings_log_var),
            axis=-1
        )
        # Add the loss (scaled by 784 for normalization).
        self.add_loss(tf.reduce_mean(kl_loss) / 784.)
        return reconstruction

# Instantiate and compile the VAE.
vae = VAE(encoder, decoder)
vae.compile(optimizer="nadam", loss="mse")




In [13]:
codings = tf.random.normal(shape=[3 * 7, codings_size])
images = decoder(codings).numpy()

In [14]:
history = vae.fit(X_train, X_train, epochs=50, batch_size=128, validation_data=(X_valid, X_valid))

Epoch 1/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0751 - val_loss: 0.0394
Epoch 2/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 10s 11ms/step - loss: 0.0385 - val_loss: 0.0366
Epoch 3/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - loss: 0.0358 - val_loss: 0.0348
Epoch 4/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - loss: 0.0345 - val_loss: 0.0340
Epoch 5/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - loss: 0.0337 - val_loss: 0.0334
Epoch 6/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 0.0331 - val_loss: 0.0331
Epoch 7/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 6s 14ms/step - loss: 0.0328 - val_loss: 0.0326
Epoch 8/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0323 - val_loss: 0.0324
Epoch 9/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - loss: 0.0321 - val_loss: 0.0320
Epoch 10/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0319 - val_loss: 0.0318
Epoch 11/50
430/430 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 0.0318 - val_loss: 0.0319
Epoch 12/50
430/430 ━━━━━━━━━━━━━━━━━━

In [15]:
codings = tf.random.normal(shape=[3 * 7, codings_size])
images = decoder(codings).numpy()